# Accelerating End-to-End Data Science Workflows # 


## 08 - แนะนำ cuDF Polars ##

**สารบัญ**

โน้ตบุ๊กนี้จะแนะนำ Polars โดยสังเขปและครอบคลุมเอ็นจิ้น GPU ใหม่ โน้ตบุ๊กนี้ครอบคลุมส่วนต่างๆ ดังนี้:

1.  [แนะนำ Polars](#Introduction-to-Polars)
    * [การติดตั้ง](#Installation)
    * [การสร้าง DataFrame](#Creating-a-DataFrame)
    * [การรันการดำเนินการพื้นฐาน](#Running-Basic-Operations)
    * [การเปรียบเทียบกับ Pandas](#Pandas-Comparison)
    * [การเปรียบเทียบกับ cuDF Pandas](#cuDF-Pandas-Comparison)
2.  [การดำเนินการ Polars พื้นฐาน](#Basic-Polars-Operations)
    * [Polars Eager Execution API Reference](#Polars-Eager-Execution-API-Reference)
    * [แบบฝึกหัดที่ 1 - โหลดข้อมูล](#Exercise-#1---Load-Data)
    * [แบบฝึกหัดที่ 2 - คำนวณอายุเฉลี่ยของประชากร](#Exercise-#2---Calculate-Average-Age-of-Population)
    * [แบบฝึกหัดที่ 3 - Group By และ Aggregation](#Exercise-#3---Group-By-and-Aggregation)
    * [แบบฝึกหัดที่ 4 - การกระจายเพศ](#Exercise-#4---Gender-Distribution)
3.  [การดำเนินการแบบ Lazy (Lazy Execution)](#Lazy-Execution)
    * [Polars Lazy Execution API Reference](#Polars-Lazy-Execution-API-Reference)
    * [กราฟการดำเนินการ (Execution Graph)](#Execution-Graph)
    * [แบบฝึกหัดที่ 5 - การสร้าง Lazy Dataframe](#Exercise-#5---Creating-a-Lazy-Dataframe)
    * [แบบฝึกหัดที่ 6 - การสร้าง Query](#Exercise-#6---Query-Creation)
4.  [cuDF Polars](#cuDF-Polars)
    * [เร่งความเร็วโค้ดก่อนหน้า](#Accelerate-Previous-Code)
    * [ตรวจสอบผลลัพธ์ข้ามเอ็นจิ้น](#Verify-Results-Across-Engines)
    * [Fallback](#Fallback)
    * [แบบฝึกหัดที่ 7 - เปิดใช้งาน GPU Engine](#Exercise-#7---Enable-GPU-Engine)

## บทนำสู่ Polars ##

Polars เป็นไลบรารีสำหรับการวิเคราะห์และจัดการข้อมูลที่ออกแบบมาสำหรับการประมวลผลข้อมูลขนาดใหญ่ (10-100GB) บน GPU ตัวเดียว และเป็นที่รู้จักในด้านความเร็วและประสิทธิภาพหน่วยความจำ แม้ว่า Pandas จะใช้การดำเนินการแบบ Eager Execution แต่ Polars ยังมีความสามารถในการดำเนินการแบบ Lazy Execution ผ่านเครื่องมือเพิ่มประสิทธิภาพคิวรีในตัว และใช้เทคนิคการเพิ่มประสิทธิภาพแบบ Zero-copy ด้วยการปรับปรุงเหล่านี้ Polars มักจะดำเนินการทั่วไปได้เร็วกว่า Pandas 5-10 เท่า และต้องการ RAM น้อยกว่า 2-4 เท่า NVIDIA นำเสนอการเร่งฮาร์ดแวร์ให้กับ Polars ผ่านเอ็นจิ้น GPU ใหม่ชื่อ cuDF Polars ซึ่งพร้อมใช้งานสำหรับการติดตั้งผ่าน pip

### การสร้าง DataFrame ###

ตอนนี้เรามาดูไวยากรณ์กัน! เราจะสร้าง DataFrame เพื่อใช้ใน Polars


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import polars as pl
import time

start_time = time.time()

polars_df = pl.read_csv('./data/uk_pop.csv')

polars_time = time.time() - start_time

print(f"Time Taken: {polars_time:.4f} seconds")

In [ ]:
polars_df.head()

### การรันการดำเนินการพื้นฐาน ###

นั่นง่ายมาก ตอนนี้ลองรันการดำเนินการบางอย่างกับชุดข้อมูลกัน! เราจะโหลดชุดข้อมูลอีกครั้งเพื่อเปรียบเทียบอย่างยุติธรรมกับ Pandas ในภายหลัง


In [ ]:
start_time = time.time()

#load data
polars_df = pl.read_csv('./data/uk_pop.csv')

# Filter for ages above 0
filtered_df = polars_df.filter(pl.col('age') > 0.0)

#Sort by name
sorted_df = filtered_df.sort('name', descending=True)

print(sorted_df.head())
polars_time = time.time() - start_time
print(f"Time Taken: {polars_time:.4f} seconds")

### Pandas Comparison ###

มาดูกันว่าใช้เวลานานแค่ไหนใน Pandas


In [ ]:
import pandas as pd
import time
start_time = time.time()
pandas_df = pd.read_csv('./data/uk_pop.csv')

filtered_df = pandas_df[pandas_df['age'] > 0.0]

sorted_df = filtered_df.sort_values(by=['name'], ascending=False)

pandas_time = time.time() - start_time
print(f"Time Taken: {pandas_time:.4f} seconds\n")

### cuDF Pandas Comparison ###

ว้าว! ใช้เวลาดำเนินการค่อนข้างนานเลย มาดูกันว่าเราจะรันให้เร็วขึ้นได้ไหมด้วย cuDF Pandas


In [ ]:
# Activate cuDF Pandas
%load_ext cudf.pandas
import pandas as pd

In [ ]:
import pandas as pd
import time
start_time = time.time()
pandas_df = pd.read_csv('./data/uk_pop.csv')

filtered_df = pandas_df[pandas_df['age'] > 0.0]

sorted_df = filtered_df.sort_values(by=['name'], ascending=False)

pandas_time = time.time() - start_time
print(f"Time Taken for cuDF Pandas: {pandas_time:.4f} seconds\n")

แม้แต่กับ cuDF Pandas เราจะเห็นว่าประสิทธิภาพช้ากว่า Polars อย่างมีนัยสำคัญ

## การดำเนินการ Polars พื้นฐาน ##

โปรดอ้างอิงคู่มือ API ต่อไปนี้เพื่อทำแบบฝึกหัดด้านล่างให้เสร็จสิ้น

1. โหลดข้อมูล
2. คำนวณอายุเฉลี่ยของประชากร
3. จัดกลุ่มตามและรวมข้อมูล
4. การกระจายเพศ


### Polars Eager Execution API Reference ###

**DataFrame**

โครงสร้างข้อมูลหลักสำหรับการดำเนินการแบบ Eager Execution ใน Polars

- `pl.DataFrame(data)`: สร้าง DataFrame จากข้อมูล
- `pl.read_csv(file)`: อ่านไฟล์ CSV เข้าสู่ DataFrame
- `pl.read_parquet(file)`: อ่านไฟล์ Parquet เข้าสู่ DataFrame

**เมธอดหลัก (Key Methods)**

- `filter(mask)`: กรองแถวตาม boolean mask
- `select(columns)`: เลือกคอลัมน์ที่ระบุ
- `with_columns(expressions)`: เพิ่มหรือแก้ไขคอลัมน์
- `group_by(columns)`: จัดกลุ่มตามคอลัมน์ที่ระบุ
- `agg(aggregations)`: ดำเนินการ Aggregations กับข้อมูลที่ถูกจัดกลุ่ม
- `sort(columns)`: เรียงลำดับข้อมูลตามคอลัมน์ที่ระบุ
- `join(other, on)`: Join กับ DataFrame อื่น

**นิพจน์ (Expressions)**

ใช้เพื่อกำหนดการดำเนินการบนคอลัมน์:

- `pl.col("column")`: อ้างอิงคอลัมน์
- `pl.lit(value)`: สร้างค่า Literal
- `pl.when(predicate).then(value).otherwise(other)`: นิพจน์แบบมีเงื่อนไข

**การดำเนินการ Series (Series Operations)**

- `series.sum()`: คำนวณผลรวมของ Series
- `series.mean()`: คำนวณค่าเฉลี่ยของ Series
- `series.max()`: ค้นหาค่าสูงสุดใน Series
- `series.min()`: ค้นหาค่าต่ำสุดใน Series
- `series.sort()`: เรียงลำดับค่าใน Series

**ประเภทข้อมูล (Data Types)**

- `pl.Int64`: จำนวนเต็ม 64 บิต
- `pl.Float64`: จำนวนทศนิยม 64 บิต
- `pl.Utf8`: สตริง
- `pl.Boolean`: บูลีน
- `pl.Date`: วันที่

**ยูทิลิตี้ (Utilities)**

- `pl.concat([df1, df2])`: รวม DataFrame
- `df.describe()`: สร้างสถิติสรุป
- `df.to_csv(file)`: เขียน DataFrame ไปยัง CSV
- `df.to_parquet(file)`: เขียน DataFrame ไปยัง Parquet

API แบบ Eager Execution จะดำเนินการทันที ทำให้สามารถเข้าถึงผลลัพธ์ได้โดยตรง เหมาะสำหรับการสำรวจข้อมูลแบบโต้ตอบและชุดข้อมูลขนาดเล็ก


### แบบฝึกหัดที่ 1 - โหลดข้อมูล ###

โหลดไฟล์ csv เข้าสู่ Dataframe โดยใช้ Polars

Click ... for solution. 

### แบบฝึกหัดที่ 2 - คำนวณอายุเฉลี่ยของประชากร ###

ตอนนี้ ให้กรองหาบุคคลที่มีอายุ 65 ปีขึ้นไป และเรียงลำดับตามอายุจากน้อยไปมาก

Click ... for solution. 

### แบบฝึกหัดที่ 3 - การจัดกลุ่มและการรวมข้อมูล (Group By and Aggregation) ###

ถัดไป ให้จัดกลุ่มตามมณฑลและคำนวณจำนวนประชากรทั้งหมดและอายุเฉลี่ย


Click ... for solution. 

### แบบฝึกหัดที่ 4 - การกระจายเพศ ###

สุดท้ายนี้ เรามาคำนวณเปอร์เซ็นต์ของผู้ชายเทียบกับผู้หญิงในข้อมูลตัวอย่างกัน


Click ... for solution. 

## การดำเนินการแบบ Lazy (Lazy Execution) ##

Polars ใช้เทคนิคที่เรียกว่า Lazy Execution ในการดำเนินการ ต่างจากการดำเนินการแบบ Eager Execution ที่การดำเนินการจะทำทันที Polars จะกำหนดและจัดเก็บการดำเนินการในกราฟการคำนวณ ซึ่งจะไม่ถูกดำเนินการจนกว่าจะมีการร้องขออย่างชัดเจน สิ่งนี้ช่วยให้ Polars สามารถเพิ่มประสิทธิภาพลำดับการดำเนินการเพื่อลดค่าใช้จ่ายในการคำนวณ และใช้เทคนิคการเพิ่มประสิทธิภาพ เช่น การใช้ตัวกรองก่อน (Predicate Pushdown), การเลือกเฉพาะคอลัมน์ที่จำเป็น (Projection Pushdown) และการดำเนินการแบบขนาน เพื่อใช้ประโยชน์จากการดำเนินการแบบ Lazy Execution ใน Polars จะใช้โครงสร้างข้อมูล "LazyFrame"

ตอนนี้ ลองรันการดำเนินการเดียวกันด้วย Lazy Execution และแสดงกราฟกัน!



### Polars Lazy Execution API Reference ###

**LazyFrame**

จุดเริ่มต้นหลักสำหรับการดำเนินการแบบ Lazy Execution ใน Polars สร้างขึ้นจาก DataFrame หรือแหล่งข้อมูล

- `pl.LazyFrame(data)`: สร้าง LazyFrame จากข้อมูล
- `df.lazy()`: แปลง DataFrame เป็น LazyFrame

**เมธอดหลัก (Key Methods)**

- `filter(predicate)`: กรองแถวตามเงื่อนไข
- `select(columns)`: เลือกคอลัมน์ที่ระบุ
- `with_columns(expressions)`: เพิ่มหรือแก้ไขคอลัมน์
- `group_by(columns)`: จัดกลุ่มตามคอลัมน์ที่ระบุ
- `agg(aggregations)`: ดำเนินการ Aggregations กับข้อมูลที่ถูกจัดกลุ่ม
- `sort(columns)`: เรียงลำดับข้อมูลตามคอลัมน์ที่ระบุ
- `join(other, on)`: Join กับ LazyFrame อื่น
- `collect()`: ดำเนินการคิวรีแบบ Lazy และส่งคืน DataFrame

**นิพจน์ (Expressions)**

ใช้เพื่อกำหนดการดำเนินการบนคอลัมน์:

- `pl.col("column")`: อ้างอิงคอลัมน์
- `pl.lit(value)`: สร้างค่า Literal
- `pl.when(predicate).then(value).otherwise(other)`: กำหนดนิพจน์แบบมีเงื่อนไข

**การดำเนินการ (Execution)**

- `collect()`: ดำเนินการและส่งคืน DataFrame
- `fetch(n)`: ดำเนินการและส่งคืน n แถวแรก
- `describe_plan()`: แสดงแผนคิวรีสำหรับข้อมูลเชิงลึกการเพิ่มประสิทธิภาพ
- `explain()`: อธิบายกระบวนการดำเนินการคิวรี

**การเพิ่มประสิทธิภาพ (Optimization)**

- `cache()`: แคชผลลัพธ์ระหว่างกลางเพื่อการเข้าถึงที่เร็วขึ้น
- `optimize()`: ใช้การเพิ่มประสิทธิภาพคิวรีเพื่อปรับปรุงประสิทธิภาพ

API แบบ Lazy ช่วยให้สามารถสร้างคิวรีที่ซับซ้อนซึ่งได้รับการเพิ่มประสิทธิภาพก่อนการดำเนินการ ทำให้ได้ประสิทธิภาพที่ดีขึ้นสำหรับชุดข้อมูลขนาดใหญ่

In [ ]:
import polars as pl
import time

start_time = time.time()

# Create a lazy DataFrame
lazy_df = pl.scan_csv('./data/uk_pop.csv')

# Define the lazy operations
lazy_result = (
    lazy_df
    .filter(pl.col('age') > 0.0)
    .sort('name', descending=True)
)

# Execute the lazy query and collect the results
result = lazy_result.collect()

print(result.head())
polars_time = time.time() - start_time
print(f"Time Taken: {polars_time:.4f} seconds")

### กราฟการดำเนินการ (Execution Graph) ###


มาดูกราฟการดำเนินการแบบไม่ปรับให้เหมาะสมกัน

In [ ]:
# Show unoptimized Graph
lazy_result.show_graph(optimized=False)

In [ ]:
# Show optimized Graph
lazy_result.show_graph(optimized=True)

อย่างที่เราเห็น ระหว่างการดำเนินการ Polars ได้รันตัวกรองอายุควบคู่ไปกับการอ่านไฟล์ CSV เพื่อประหยัดเวลา! การปรับปรุงประสิทธิภาพประเภทนี้เป็นส่วนหนึ่งของเหตุผลที่ทำให้ Polars เป็นเครื่องมือ Data Science ที่ทรงพลัง


### แบบฝึกหัดที่ 5 - การสร้าง Lazy Dataframe ###

อันดับแรก มาโหลดไฟล์ CSV เป็น Lazy Dataframe กัน

Click ... for solution. 

### แบบฝึกหัดที่ 6 - การสร้าง Query ###

ตอนนี้ มาสร้างคิวรีเพื่อค้นหา 5 ชื่อที่พบบ่อยที่สุดสำหรับบุคคลที่มีอายุต่ำกว่า 30 ปี

Click ... for solution. 

## cuDF Polars ##

cuDF Polars ถูกสร้างขึ้นโดยตรงใน Polars Lazy API ข้อกำหนดเดียวคือการส่ง `engine="gpu"` ไปยังการดำเนินการ `collect` Polars ยังอนุญาตให้กำหนดอินสแตนซ์ของ GPU Engine เพื่อการปรับแต่งที่มากขึ้น!


In [ ]:
lazy_df = pl.scan_csv('./data/uk_pop.csv').collect(engine="gpu")

ตอนนี้เรามาลองกำหนดอ็อบเจกต์เอ็นจิ้นของเราเองกัน!

In [ ]:
import polars as pl
import time

gpu_engine = pl.GPUEngine(
    device=0, # This is the default
    raise_on_fail=True, # Fail loudly if we can't run on the GPU.
)

In [ ]:
lazy_df = pl.scan_csv('./data/uk_pop.csv').collect(engine=gpu_engine)

เมื่อ GPU ได้รับการวอร์มอัพแล้ว มาลองเร่งความเร็วโค้ดเดิมกัน! สังเกตว่าเราได้เพิ่มพารามิเตอร์ `engine` ในการเรียกใช้ `.collect()`


### เร่งความเร็วโค้ดก่อนหน้า ###

In [ ]:
start_time = time.time()

# Create a lazy DataFrame
lazy_df = pl.scan_csv('./data/uk_pop.csv')

# Define the lazy operations
lazy_result = (
    lazy_df
    .filter(pl.col('age') > 0.0)
    .sort('name', descending=True)
)

# Switch to gpu_engine
result = lazy_result.collect(engine=gpu_engine)

print(result.head())
polars_time = time.time() - start_time
print(f"Time Taken: {polars_time:.4f} seconds")

### ตรวจสอบผลลัพธ์ข้ามเอ็นจิ้น ###

เราจะรู้ได้อย่างไรว่าผลลัพธ์ที่ได้จากเอ็นจิ้น CPU และ GPU นั้นเหมือนกัน? โชคดีที่ Polars มีโมดูลการทดสอบในตัว เราสามารถรันคิวรีเดียวกันบนทั้งสองเอ็นจิ้นและเปรียบเทียบผลลัพธ์ได้!


In [ ]:
from polars.testing import assert_frame_equal

# Run on the CPU
result_cpu = lazy_result.collect()

# Run on the GPU
result_gpu = lazy_result.collect(engine="gpu")

# assert both result are equal - Will error if not equal, return None otherwise
if (assert_frame_equal(result_gpu, result_cpu) == None):
    print("The test frames are equal")

### Fallback ###

จะเกิดอะไรขึ้นเมื่อการดำเนินการไม่ได้รับการรองรับ?


In [ ]:
result = (
    lazy_df
    .with_columns(pl.col('age').rolling_mean(window_size=7).alias('age_rolling_mean'))
    .filter(pl.col('age') > 0.0)  
    .collect(engine=gpu_engine)
)
print(result[::7])

เราเริ่มต้นสร้าง GPU engine ด้วย `raise_on_fail=True` เพื่อให้แน่ใจว่าการดำเนินการทั้งหมดทำงานบน GPU แต่ดังที่เราเห็น การดำเนินการ `rolling_mean` ไม่รองรับในปัจจุบัน ซึ่งทำให้คิวรีไม่สามารถดำเนินการได้ หากต้องการเปิดใช้งานการกลับไปใช้ (fallback) เราสามารถเปลี่ยนพารามิเตอร์ `raise_on_fail` เป็น `False` ได้

In [ ]:
gpu_engine_with_fallback = pl.GPUEngine(
    device=0, # This is the default
    raise_on_fail=False, # Fallback to CPU if we can't run on the GPU (this is the default)
)

ตอนนี้เรามาลองคิวรีนี้อีกครั้ง

In [ ]:
result = (
    lazy_df
    .with_columns(pl.col('age').rolling_mean(window_size=7).alias('age_rolling_mean'))
    .filter(pl.col('age') > 0.0)  
    .collect(engine=gpu_engine_with_fallback)
)
print(result[::7])

### แบบฝึกหัดที่ 7 - เปิดใช้งาน GPU Engine ###

โค้ดด้านล่างคำนวณค่าเฉลี่ยของละติจูดและลองจิจูดสำหรับแต่ละมณฑล ลองเปิดใช้งาน GPU Engine สำหรับคิวรีนี้กัน!


In [ ]:
# Create the lazy query with column pruning
lazy_query = (
    lazy_df
    .select(["county", "lat", "long"])  # Column pruning: select only necessary columns
    .group_by("county")
    .agg([
        pl.col("lat").mean().alias("avg_latitude"),
        pl.col("long").mean().alias("avg_longitude")
    ])
    .sort("county")
)

# Execute the query
result = lazy_query.collect()

print("\nAverage latitude and longitude for each county:")
print(result.head())  # Display first few rows

Click ... for solution. 

In [ ]:
import IPython
app = IPython.Application.instance()
app.kernel.do_shutdown(True)

**เยี่ยมมาก!** ไปยัง [โน้ตบุ๊กถัดไป](1-09_dask-cudf.ipynb) กันเลย